In [ ]:
%load_ext autoreload
%autoreload 2
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
from braininversion.BiotSolver import solve_biot
from braininversion.Optimization import optimize_biot_force
from braininversion.Optimization import optimize_darcy_source, compute_minimization_target, update_expression_time
from braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                            plot_pressures_and_forces_cross_section,
                            extract_cross_section, style_dict)


T = 1.0           # final time
num_steps = 20    # number of time steps
dt = T / num_steps
times = np.linspace(dt, T, num_steps)

# material parameter
kappa = 1e-17       # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity
c = 2*1e-8         # storage coefficent
alpha = 1.0         # Biot-Willis coefficient

# Biot material parameters
E = 1500.0          # Young modulus
nu = 0.479         # Poisson ratio

material_parameter = dict()
material_parameter["c"] = c
material_parameter["K"] = K
material_parameter["lmbda"] = nu*E/((1.0-2.0*nu)*(1.0+nu)) 
material_parameter["mu"] = E/(2.0*(1.0+nu))
material_parameter["alpha"] = alpha

mesh = Mesh(IntervalMesh(100,0.0,0.1))
bm = MeshFunction("size_t", mesh, 0)
left = CompiledSubDomain("near(x[0], 0.0)")
right = CompiledSubDomain("near(x[0], 0.1)")
left.mark(bm, 1)
right.mark(bm, 2)


mmHg2Pa = 132.32
freq = 1.0 
A = 2*mmHg2Pa
p_skull = Expression("A*sin(2*pi*f*t)*(0.8 + 2*x[0])", A=A,f=freq,t=0,degree=2)

n = FacetNormal(mesh)
boundary_conditions_u = {1:{"Neumann":n*p_skull},
                         2:{"Neumann":n*p_skull},
                         #1:{"Dirichlet":OneDimZeroVec(),
                         #2:{"Dirichlet":Constant([0.0])}
                        } 
                         
boundary_conditions_p = {#1:{"Dirichlet":p_skull},
                         1:{"Neumann":Constant(0.0)},
                         #2:{"Dirichlet":p_skull},
                         2:{"Neumann":Constant(0.0)},
                         }
u_nullspace = True

time_dep_expr = [p_skull]

minimization_target = {#"ds": { 1: lambda x: (x - p_ventricle)**2,
                       #       2: lambda x: (x - p_skull)**2},
                       #"dS":{"everywhere":laplace}
                        "dx":{"everywhere": lambda x: (x - p_skull)**2}
                      }



In [ ]:
solution = optimize_biot_force(mesh, material_parameter, times, minimization_target,
                              bm, boundary_conditions_p,
                              bm, boundary_conditions_u,
                              time_dep_expr=time_dep_expr,
                              optimization_parameters={"maxiter":1, "jtol":1e-12},
                              u_nullspace=u_nullspace, theta=1.0)

In [ ]:
opt_ctrl, opt_solution, initial_solution = solution
displ = [s.split()[0] for s in opt_solution]
total_pressure = [s.split()[1] for s in opt_solution]
pressure = [s.split()[2] for s in opt_solution]
initial_pressure = [s.split()[2] for s in initial_solution]

V = FunctionSpace(mesh, "CG", 1)
lmbda_div_u = [project(material_parameter["lmbda"]*div(u), V) for u in displ]

x_coords = np.linspace(0.0, 0.1, 100)
slice_points = [Point(x, 0.0) for x in x_coords]


In [ ]:
pressures = {"negative_total_pressure" : extract_cross_section(total_pressure, slice_points)/mmHg2Pa,
             "fluid_pressure" : extract_cross_section(pressure, slice_points)/mmHg2Pa,
             "initial_fluid_pressure" : extract_cross_section(initial_pressure, slice_points)/mmHg2Pa,
             "lambda_div_u"   : extract_cross_section(lmbda_div_u, slice_points)/mmHg2Pa,
             #"prescribed outer pressure"   : extract_cross_section(p_skull, slice_points, times)/mmHg2Pa,
            }


forces = {"displacement [m]": extract_cross_section(displ, slice_points),
          "f_opt [N]": extract_cross_section(opt_ctrl, slice_points)}

style_dict["negative_total_pressure"] = {"ls":":", "lw":3, "color":"firebrick"}
style_dict["initial_fluid_pressure"] = {"ls":":", "lw":3, "color":"orange"}
style_dict["fluid_pressure"] = {"ls":"-.", "lw":3, "color":"green"}
style_dict["displacement [m]"] = {"ls":"-.", "lw":3, "color":"green"}
style_dict["f_opt [N]"] = {"ls":"-.", "lw":3, "color":"blue"}
style_dict["prescribed outer pressure"] = {"ls":"-.", "lw":1, "color":"navy"}


In [ ]:
for i in [2,4,6,8]: 
    plot_pressures_and_forces_cross_section(pressures, forces, i, x_coords)
    plt.suptitle(f"t = {times[i]:.3f} s")

In [ ]:
for i in [20, 40 ,60, 80]:
    plot_pressures_and_forces_timeslice(pressures, forces, i, times)
    plt.suptitle(f"Point: ({slice_points[i].x():.3f}, {slice_points[i].y():.3f})")